<a href="https://colab.research.google.com/github/francescogrillea/AnomalyDetectionTFLite/blob/main/AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import e Costanti

In [13]:
!cat ./drive/MyDrive/Colab*/anomaly_detection_data/res*

Tue Nov 23 15:12:43 2021
Architecture: LSTM_1L
Channel: A-1
TensorFlow execution time: 0.6743566989898682s
TensorFlowLite execution time: 0.0004329681396484375s
TensorFlow predictions	TFLite predictions	Distance
0.9981911182403564	0.9926959276199341	0.005495190620422363
0.9989597797393799	0.9959942698478699	0.0029655098915100098
0.9989480376243591	0.9968134760856628	0.002134561538696289
0.9989720582962036	0.9970735907554626	0.0018984675407409668
0.9992333054542542	0.9978994131088257	0.0013338923454284668
0.9990507364273071	0.9971656203269958	0.0018851161003112793
0.9991313219070435	0.9977410435676575	0.0013902783393859863
0.9983036518096924	0.9939520359039307	0.004351615905761719
0.9991016983985901	0.9988558292388916	0.00024586915969848633
0.9989663362503052	0.996996283531189	0.001970052719116211
Tue Nov 23 15:13:41 2021
Architecture: LSTM_2L
Channel: A-1
TensorFlow execution time: 1.1536469459533691s
TensorFlowLite execution time: 0.000396728515625s
TensorFlow predictions	TFLite predi

In [10]:
!rm ./drive/MyDrive/Colab*/anomaly_detection_data/res*

rm: cannot remove './drive/MyDrive/Colab*/anomaly_detection_data/res*': No such file or directory


In [8]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
import logging

PROJECT_NAME = 'anomaly_detection'
PROJECT_PATH = './drive/MyDrive/Colab Notebooks/anomaly_detection_data/'
DATASET_PATH = PROJECT_PATH+'Dataset/'
ARCHITECTURE_TYPE = 'LSTM_2L/'
TF_MODELS_PATH = PROJECT_PATH+'TF_Models/'+ARCHITECTURE_TYPE
TFLite_MODELS_PATH = PROJECT_PATH+'TFLite_Models/'+ARCHITECTURE_TYPE
CHANNEL_NAME = 'A-1'

logging.basicConfig(filename="./drive/MyDrive/Colab Notebooks/anomaly_detection_data/results.log", level=logging.INFO, format='%(message)s')
logging.info('{}\nArchitecture: {}\nChannel: {}'.format(time.ctime(), ARCHITECTURE_TYPE[:-1], CHANNEL_NAME))

# Load Model and Dataset

In [9]:
def load_tf_model(channel_name):
  if not channel_name.endswith('.h5'):
    channel_name = channel_name+'.h5'
  tf_model = keras.models.load_model(TF_MODELS_PATH+channel_name)
  return tf_model


def load_dataset(channel_name):
  if not channel_name.endswith('.npy'):
    channel_name = channel_name+'.npy'
  x = np.array([np.load(DATASET_PATH+'test/'+channel_name)])
  y = np.array([])
  return x,y

# Prediction su TF Model

In [10]:
#load dataset
tf_input_data, tf_output_data = load_dataset(CHANNEL_NAME)

model = load_tf_model(CHANNEL_NAME)
start_time = time.time()
tf_output_data = model.predict(tf_input_data)
delta_time = time.time()-start_time
logging.info('TensorFlow execution time: {}s'.format(delta_time))
print('Library: {}\nChannel: {}\nPredictions: {}\nTime elapsed: {}s'.format('TensorFlow', CHANNEL_NAME, tf_output_data, delta_time))

Library: TensorFlow
Channel: A-1
Predictions: [[0.97624755 0.97461784 0.97506773 0.9697092  0.9722527  0.9748879
  0.97258246 0.97561336 0.96816814 0.96876484]]
Time elapsed: 1.1536469459533691s


# Conversione dei modelli da .h5 a .tflite

In [17]:
channels = [f for f in os.listdir(TF_MODELS_PATH)]

for model_filename in channels:
  if not model_filename.startswith(CHANNEL_NAME):
    continue

  tf_model = load_tf_model(model_filename)
  converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
  converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
  optimizations = [tf.lite.Optimize.DEFAULT]
  #WARNING:absl:Found untraced functions such as lstm_cell -> e' un warning che dipende da come è stato salvato il modello, tuttavia non sembra esserci una precisa soluzione
  tflite_model = converter.convert()

  #generate TFLite model name
  channel_name = model_filename[:-3]
  tflite_model_filename = channel_name+'.tflite'
  
  TFLite_MODEL_FILE = TFLite_MODELS_PATH+tflite_model_filename
  with open(TFLite_MODEL_FILE, 'wb') as f:
    f.write(tflite_model)
    print('Model for channel '+channel_name+' converted')
    #break

INFO:tensorflow:Assets written to: /tmp/tmpwxh3d1f8/assets


INFO:tensorflow:Assets written to: /tmp/tmpwxh3d1f8/assets


Model for channel A-1 converted


# Inferenza su TFLite

In [11]:
#load dataset
x_data, tflite_output_data = load_dataset(CHANNEL_NAME)

interpreter = tf.lite.Interpreter(model_path=TFLite_MODELS_PATH+CHANNEL_NAME+'.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#preprocessing input data
input_shape = input_details[0]['shape']

start_time = time.time()
#per ogni riga del dataset, creo un tensore e invoco il modello
for data in x_data[0]:
  input_data = np.array([[data]], dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
    
  #run inference
  start_time = time.time()
  interpreter.invoke()
  stop_time = time.time()
  
  #recupero l'output prodotto dal modello
  output_data = interpreter.get_tensor(output_details[0]['index'])
  tflite_output_data = np.append(tflite_output_data, output_data)

delta_time = time.time() - start_time
logging.info('TensorFlowLite execution time: {}s'.format(delta_time))
print('Library: {}\nChannel: {}\nPredictions: {}\nTime elapsed: {}s'.format('TensorFlow Lite', CHANNEL_NAME, tflite_output_data, delta_time))

Library: TensorFlow Lite
Channel: A-1
Predictions: [0.59931397 0.61202842 0.64623725 ... 0.5906468  0.61450905 0.57663977]
Time elapsed: 0.000396728515625s


In [12]:
#confronto tra i risultati prodotti dal modello TF (sinistra) rispetto al modello TFLite (destra)
logging.info('TensorFlow predictions\tTFLite predictions\tDistance')
for y_tf, y_tflite in zip(tf_output_data[0], tflite_output_data):
  logging.info('{}\t{}\t{}'.format(y_tf, y_tflite, y_tf-y_tflite))
  print('{} {} {}'.format(y_tf, y_tflite, y_tf-y_tflite))

0.9762475490570068 0.5993139743804932 0.37693357467651367
0.9746178388595581 0.6120284199714661 0.36258941888809204
0.9750677347183228 0.6462372541427612 0.3288304805755615
0.9697092175483704 0.5865984559059143 0.38311076164245605
0.9722527265548706 0.624273955821991 0.34797877073287964
0.9748879075050354 0.627856969833374 0.3470309376716614
0.9725824594497681 0.6239501237869263 0.3486323356628418
0.9756133556365967 0.5906468033790588 0.38496655225753784
0.9681681394577026 0.6145090460777283 0.35365909337997437
0.9687648415565491 0.5766397714614868 0.39212507009506226
